In [1]:
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from base_module import (
    get_preprocessed_data,
    train_and_evaluate,
    save_model_results,
    SVM_PARAMS,
    RANDOM_STATE
)
import time

ALGORITHM_NAME = "Spearman_Filter"
# !! IMPORTANT: Update this path to your local file path !!
DATA_PATH = r'D:\ACTUAL STUDY MATERIAL\IPD\Data\diabetic_data.csv'

# We will select features with an absolute Spearman correlation > 0.05
# This is a common threshold for this kind of filter.
CORR_THRESHOLD = 0.05

print("="*70)
print(f"FEATURE SELECTION: {ALGORITHM_NAME}")
print("="*70)

FEATURE SELECTION: Spearman_Filter


In [2]:
print("\n--- Loading and Preprocessing Data ---")

X_train_encoded, X_test_encoded, y_train, y_test, \
    all_encoded_features, all_original_features, feature_mapping = \
    get_preprocessed_data(DATA_PATH)

print(f"\nStarting with {len(all_encoded_features)} encoded features")


--- Loading and Preprocessing Data ---
Loading data from D:\ACTUAL STUDY MATERIAL\IPD\Data\diabetic_data.csv...


d:\ACTUAL STUDY MATERIAL\IPD\src\base_module.py:31: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, na_values='?')


Cleaned data shape: (98053, 43)
Engineering features...
Engineered data shape: (98053, 41)

Preprocessing data...
Ordinal Encoding 31 features...
Encoded features: 40
  - Numeric: 9
  - Categorical (Ordinal): 31

Starting with 40 encoded features


In [3]:
print(f"\n--- Applying {ALGORITHM_NAME} (Scipy Implementation) ---")
print(f"Calculating Spearman correlation for {len(all_encoded_features)} features...")
start_time = time.time()

correlations = {}
# Calculate correlation for each feature against the training target
for i in range(X_train_encoded.shape[1]):
    feat_name = all_encoded_features[i]
    feat_vector = X_train_encoded[:, i]
    
    # Use spearmanr for rank correlation
    corr, p_value = spearmanr(feat_vector, y_train)
    correlations[feat_name] = corr

elapsed = (time.time() - start_time)
print(f"✓ Correlation calculation complete in {elapsed:.2f}s")

# --- 2. Rank and Select Features ---
corr_series = pd.Series(correlations).sort_values(key=abs, ascending=False)

# Filter by the threshold
selected_features = corr_series[corr_series.abs() > CORR_THRESHOLD].index.tolist()

# --- 3. Filter Datasets ---
# Get the column indices in the *original* encoded data
original_indices_mapping = {feat: i for i, feat in enumerate(all_encoded_features)}
final_selected_original_indices = [original_indices_mapping[feat] for feat in selected_features]

X_train_selected = X_train_encoded[:, final_selected_original_indices]
X_test_selected = X_test_encoded[:, final_selected_original_indices]

print(f"\nFeature Selection Results:")
print(f"  Selected: {len(selected_features)} features (Threshold: |r| > {CORR_THRESHOLD})")
print(f"  Eliminated: {len(all_encoded_features) - len(selected_features)} features")


--- Applying Spearman_Filter (Scipy Implementation) ---
Calculating Spearman correlation for 40 features...
✓ Correlation calculation complete in 0.22s

Feature Selection Results:
  Selected: 4 features (Threshold: |r| > 0.05)
  Eliminated: 36 features


C:\Users\Soham\AppData\Local\Temp\ipykernel_24048\235824161.py:12: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  corr, p_value = spearmanr(feat_vector, y_train)


In [4]:
print(f"\n--- Training Model on {ALGORITHM_NAME}-Selected Features ---")

clf, y_pred, report = train_and_evaluate(
    X_train_selected, X_test_selected,
    y_train, y_test,
    **SVM_PARAMS
)

print(f"\nModel Performance:")
print(f"  Accuracy: {report['accuracy']:.4f}")
print(f"  Precision (class 0): {report['0']['precision']:.4f}")
print(f"  Recall (class 0): {report['0']['recall']:.4f}")
print(f"  F1-score (class 0): {report['0']['f1-score']:.4f}")
print(f"  Precision (class 1): {report['1']['precision']:.4f}")
print(f"  Recall (class 1): {report['1']['recall']:.4f}")
print(f"  F1-score (class 1): {report['1']['f1-score']:.4f}")


--- Training Model on Spearman_Filter-Selected Features ---

Training SVM classifier...


d:\ACTUAL STUDY MATERIAL\IPD\.venv\Lib\site-packages\sklearn\svm\_base.py:305: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Evaluating...

Model Performance:
  Accuracy: 0.6424
  Precision (class 0): 0.9056
  Recall (class 0): 0.6663
  F1-score (class 0): 0.7678
  Precision (class 1): 0.1476
  Recall (class 1): 0.4541
  F1-score (class 1): 0.2228


In [5]:
print(f"\n--- Saving Results ---")

save_model_results(
    algorithm_name=ALGORITHM_NAME,
    selected_encoded_features=selected_features,
    all_encoded_features=all_encoded_features,
    all_original_features=all_original_features,
    feature_mapping=feature_mapping,
    report=report,
    results_csv='study_results.csv'
)

print("\n" + "="*70)
print(f"{ALGORITHM_NAME} COMPLETE")
print("="*70)


--- Saving Results ---

Saving results for: Spearman_Filter
✓ Appended to study_results.csv
✓ Saved to spearman_filter_results.json

  Original: 4/40 features
  Encoded:  4/40 features
  Accuracy: 0.6424
  F1 (class 1): 0.2228

Spearman_Filter COMPLETE
